In [785]:
import math
import copy
import random
import functools
import numpy as np
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt

In [786]:
# Define the blueprint of the city object (gene)
class city:

  def __init__(self, name, x, y):
    self._name = name
    self._x = x
    self._y = y
    self._visited = False

  @property
  def name(self):
    return self._name

  @name.setter
  def name(self, name):
    self._name = name


  @property
  def x(self):
    return self._x

  @x.setter
  def x(self, x):
    self._x = x


  @property
  def y(self):
    return self._y

  @y.setter
  def y(self, y):
    self._y = y


  @property
  def visited(self):
    return self._visited

  @visited.setter
  def visited(self, visited):
    self._visited = visited

In [787]:
# Define the blueprint of the path object
class path:

  def __init__(self, fitness, cost=0.0, cities=[]):
    self._cities = cities
    self._fitness = fitness
    self._cost = cost

  @property
  def fitness(self):
    return self._fitness


  @fitness.setter
  def fitness(self, fitness):
    self._fitness = fitness


  @property
  def cost(self):
    return self._cost


  @cost.setter
  def cost(self, cost):
    self._cost = cost


  @property
  def cities(self):
    return self._cities


  @cities.setter
  def genes(self, cities):
    self._cities = cities

## **1. Reading the Data**

In [788]:
df = pd.read_csv("/content/Data set CSV.csv")

In [789]:
df.head()

,City,x,y
0,1,5.500000e-08,9.860000e-09
1,2,-2.887330e+01,-7.980000e-08
2,3,-7.929160e+01,-2.140330e+01
3,4,-1.465770e+01,-4.338960e+01
4,5,-6.474730e+01,2.189820e+01


In [790]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   City    15 non-null     int64  
 1   x       15 non-null     float64
 2   y       15 non-null     float64
dtypes: float64(2), int64(1)
memory usage: 492.0 bytes


## **2. Mapping the Data To Objects**

In [791]:
cities_objects = []     # Initialize an empty cities list

# Iterate through each row (city data) in the dataframe
for index, row in df.iterrows():
  x, y = row['x'], row['y']         # Get city coordinates
  name = row['City']                # Get city name
  city_obj = city(name, x, y)       # Create city object from the data
  cities_objects.append(city_obj)   # Append the city object to the cities

In [792]:
cities_objects

## **3. Get Euclidean Distance Between 2 Cities**

In [793]:
def get_euc_dist(city_1, city_2):
  '''
  Function To get the euclidean (shortest) distance between 2 cities.

    Args:
      city_1 (city) : Source City.
      city_2 (city) : Distination City.

    Returns:
      euc_dist (float) : The shortest distance between both cities (euclidean distance).
  '''

  euc_dist = np.sqrt((city_1.x - city_2.x)**2 + (city_1.y - city_2.y)**2)

  return euc_dist

## **4. Create The Distance & Initial Pheromone Matrix**

In [794]:
def get_distance_matrix(cities_objects):
  '''
  Function To get the distance matrix, i.e euclidean (shortest) distance between 2 pair of cities.

    Args:
      cities_objects (list) : List of all cities as city object in our dataframe.

    Returns:
      distance_matrix (np.array) : The distance matrix between each and other cities.
  '''

  n_cities = len(cities_objects)
  distance_matrix = np.zeros(shape=(n_cities, n_cities))

  for city_1 in cities_objects:

      for city_2 in cities_objects:

         distance_matrix[int(city_1.name)-1][int(city_2.name)-1] = get_euc_dist(city_1, city_2)


  return distance_matrix

In [795]:
def initialize_pheromone_matrix(cities_objects, tau_init) -> np.array :
  '''
  Function To initialize the pheromone matrix by some value tau_init.

    Args:
      cities_objects (list) : List of all cities as city object in our dataframe.
      tau_init (float) : Initial Pheromone value.

    Returns:
      pheromone_matrix (np.array) : The matrix with initial pheromone values between each and other cities.
  '''

  n_cities = len(cities_objects)
  pheromone_matrix = np.ones(shape=(n_cities, n_cities)) * tau_init

  return pheromone_matrix

In [796]:
distance_matrix = get_distance_matrix(cities_objects)

In [797]:
pheromone_matrix = initialize_pheromone_matrix(cities_objects, tau_init=0.0)

## **5. Generate the Initial Population (Initialization Phase)**

In [798]:
# Ant Colony Algorithm parameters:
population_size = 100
n_iterations = 100

In [799]:
def get_cost(path, distance_matrix):
  '''
  Function To get the cost for a certain path of cities.

    Args:
      path (list) : List of all cities' names in the path, each city name is casted to integer then decremented by 1.
      distance_matrix (np.array) : An array of distance between each and every city in the dataframe, in the shape of (n_cities. n_cities), where n_cities is the number of cities in the dataframe.

    Returns:
      cost (float): The total cost for the path.
  '''

  cost = 0

  # Get the distance between each 2 consecutive cities
  for idx, city_obj in enumerate(path[:-1]):
      curr_city, next_city = city_obj, path[idx+1]
      cost += distance_matrix[curr_city][next_city]

  # Get Distance Between The Last & First City
  first_city, last_city = path[0], path[-1]
  cost += distance_matrix[first_city][last_city]

  return cost

In [800]:
def generate_initial_population(population_size, cities):
  '''
  Function To generate an initial population with certain size.

    Args:
      population_size (int) : The desired size of the population.
      cities (list): List of all city objects in the dataframe.

    Returns:
      initial_population (list): The list containing initial population of paths (routes).
  '''

  initial_population = []         # Initialize a list to store the initial population
  n_cities = len(cities)          # Get number of cities

  distance_matrix = get_distance_matrix(cities)       # Get the distance matrix for cities
  cities_sorted = sorted(cities, key=lambda city:city.name)


  # Generate the initial population
  for path_idx in range(population_size):

    chosen_route = np.random.permutation(n_cities)     # Get random combination of indices for the path
    cost = get_cost(chosen_route, distance_matrix)      # Get the cost for that chosen path

    # Store Path Data
    fitness = 1 / cost        # Get the fitness
    cities = [cities_sorted[idx] for idx in chosen_route]     # Get the cities objects of that path
    path_obj = path(fitness=fitness, cost=cost, cities=cities)      # Form the path object

    # Add New Path To our Population
    initial_population.append(path_obj)


  return initial_population

In [801]:
initial_population = generate_initial_population(population_size, cities_objects)

In [802]:
print('Initial Population Size:', len(initial_population))

Initial Population Size: 100


## **6. Construction Phase**

In [803]:
def construction(population : list, pheromone_matrix : np.array, evap_rate : float, alpha : float, beta : float):
  '''
  Function To update the pheromone matrix based on a given population.

    Args:
      population (list) : List of the current population (various paths), that will be used to update the pheromone matrix.
      pheromone_matrix (np.array) :  The cumulative pheromone matrix.
      evap_rate (float) : The Evaporation Rate.
      alpha (float) : Parameter that determines the exploration rate.
      beta (float) : Parameter that determines the exploitation rate.

    Returns:
      pheromone_matrix (np.array) :  The updated cumulative pheromone matrix.
  '''

  # Iterate through each path we have in our population
  for path_obj in population:

    cities = path_obj.cities      # Get the cities objects of that path
    fitness = path_obj.fitness    # Get the fitness of that path (1 / cost)

    # Update the pheromone matrix based on that path
    for idx, city in enumerate(cities[:-1]):

      city_1, city_2 = int(city.name)-1, int(cities[idx+1].name)-1
      pheromone_matrix[city_1, city_2] = ((1-evap_rate) * pheromone_matrix[city_1, city_2]) + fitness

    # Take into account the fitness between the last city and first city
    city_1, city_2 = int(cities[-1].name)-1, int(cities[0].name)-1
    pheromone_matrix[city_1, city_2] =  ((1-evap_rate) * pheromone_matrix[city_1, city_2]) + fitness

  return pheromone_matrix

In [804]:
pheromone_matrix = construction(initial_population, pheromone_matrix, evap_rate=0.0, alpha=0.01, beta=0.01)
print(pheromone_matrix)

[[0.         0.01071283 0.01178956 0.00823059 0.01311498 0.00772436
  0.00400954 0.00932138 0.01463528 0.01191313 0.01099958 0.01347038
  0.01131946 0.01178195 0.01176451]
 [0.01389556 0.         0.00867418 0.01356559 0.00955375 0.00440118
  0.01688319 0.01805206 0.00761272 0.00438865 0.01196558 0.01191883
  0.01182913 0.01023588 0.00781124]
 [0.01555555 0.00947641 0.         0.0134011  0.01537739 0.01547216
  0.00898965 0.         0.01325559 0.00737002 0.0134448  0.0091504
  0.01021125 0.01118256 0.00790066]
 [0.01661255 0.01571783 0.00712586 0.         0.01148683 0.01418997
  0.01301235 0.01637126 0.01515772 0.0062909  0.00801577 0.00430801
  0.00743929 0.00309512 0.01196407]
 [0.01037496 0.00614048 0.01828845 0.0147253  0.         0.01145722
  0.00706307 0.00868583 0.01226634 0.01788364 0.00872332 0.00762272
  0.01373667 0.00731971 0.00649983]
 [0.01370152 0.00726999 0.00459849 0.01451616 0.00703659 0.
  0.00890472 0.00784139 0.01051282 0.00411975 0.01727984 0.01405364
  0.01644919 

## **7. Get the Probabilities**

In [805]:
def get_probabilities(starting_city : city, remaining_cities : list, pheromone_matrix : np.array, distance_matrix :np.array, alpha : float, beta : float):
  '''
  Function To get the probabilities for each city from some certain starting city.

    Args:
      starting_city (city) : The random starting city.
      remaining_cities (list) :  All the other non-chosen / non-visited cities.
      pheromone_matrix (np.array) :  The cumulative pheromone matrix.
      distance_matrix (np.array) :  The distance matrix between cities.
      alpha (float) : Parameter that determines the exploration rate.
      beta (float) : Parameter that determines the exploitation rate.

    Returns:
      probabilities_dict (dict) : Dictionary with the corresponsing probability to each city.
  '''

  probabilities_dict = {}       # Dictionary that will hold the corresponsing probability to each city
  total_prob = 0.0              # Initialize the cumulative probability

  # Get the probability for each remaining city
  for city_obj in remaining_cities:

    starting_city_idx, potential_city_idx = (int(starting_city.name) - 1) , (int(city_obj.name) - 1)    # Get the indicies of the cities
    prob = (pheromone_matrix[starting_city_idx, potential_city_idx]**alpha) * ((1 / distance_matrix[starting_city_idx, potential_city_idx])**beta)
    probabilities_dict[potential_city_idx] = prob

    total_prob += prob  # Accumulate total probabilities

  # Normalize the probabilities
  probabilities_dict = {key : (val / total_prob) for key, val in probabilities_dict.items()}

  return probabilities_dict

In [806]:
n_cities = len(cities_objects)
random_idx = random.randint(0, n_cities)
starting_city = [city for city in cities_objects if int(city.name) == (random_idx+1)]
remaining_cities = [city for city in cities_objects if int(city.name) != (random_idx+1)]

probabilities_dict = get_probabilities(starting_city[0], remaining_cities, pheromone_matrix, distance_matrix, alpha=0.01, beta=0.001)
print(probabilities_dict)

{0: 0.07112021139094203, 1: 0.07167342821642794, 2: 0.07095714907970566, 3: 0.07144148669174238, 4: 0.07148491373763621, 5: 0.07141630687717061, 6: 0.07183061661767351, 7: 0.07130361724201889, 8: 0.07135205656609557, 9: 0.07104816304437231, 11: 0.07176535920460705, 12: 0.07116211975067237, 13: 0.07175219898552504, 14: 0.07169237259541048}


## **8. Get the Cumulative Density Function**

In [807]:
def get_cdf(probabilities_dict : dict):
  '''
  Function To convert the given probabilities to a cumulative distributive function.

    Args:
      probabilities_dict (dict) : Initial Dictionary with the corresponsing probability to each city.

    Returns:
      probabilities_dict (dict) : Final Dictionary with the corresponsing cumulative distributive function to each city.
  '''

  # Arrange the probabilities in ascending order
  probabilities_dict = dict(sorted(probabilities_dict.items(), key = lambda prob_item : prob_item[1]))
  #print(probabilities_dict)


  # Get the cumulative probabilities
  cum_prob = 0.0
  for city, prob in probabilities_dict.items():
    cum_prob += prob
    probabilities_dict[city] = cum_prob

  return probabilities_dict

In [808]:
cdf_map = get_cdf(probabilities_dict)
print(cdf_map)

{2: 0.07095714907970566, 9: 0.14200531212407796, 0: 0.21312552351502, 12: 0.2842876432656924, 7: 0.3555912605077113, 8: 0.42694331707380684, 5: 0.49835962395097744, 3: 0.5698011106427199, 4: 0.6412860243803561, 1: 0.7129594525967841, 14: 0.7846518251921946, 13: 0.8564040241777195, 11: 0.9281693833823266, 6: 1.0}


## **9. Get the Fittest Solution**

In [809]:
def get_fittest(population):
  '''
  Function To get the fittest path i.e path with the least cost.

    Args:
      population (list) : List of the final population (various paths), to choose the best fit from.

    Returns:
      fittest_solution (path) : Path with the highest fitness (lowest cost).
  '''

  # Sort the population by fitness (from fittest to least fit)
  sorted_population = sorted(population, key=lambda solution:solution.fitness, reverse=True)
  fittest_solution = sorted_population[0]     # Get the fittesst

  return fittest_solution

In [810]:
fittest_solution = get_fittest(initial_population)
print(fittest_solution.fitness)

0.002006120615973338


## **10. Ant Colony Algorithm (ACO)**

In [824]:
def aco_algorithm(cities_objects : list, population_size : int, n_iterations : int, evap_rate : float, alpha : float, beta : float):
  '''
  Function To perform Ant Colony Algorithm to solve Travelling Salesman Problem (TSP).

    Args:
      cities_objects (list) : List of unique cities in our dataframe as city objects.
      population_size (int) : The desired population size.
      n_iterations (int) : The desired maximum number of iterations.
      evap_rate (float) : The Evaporation Rate.
      alpha (float) : Parameter that determines the exploration rate.
      beta (float) : Parameter that determines the exploitation rate.

    Returns:
      new_population (list) : List of the final population (various paths), to choose the best fit from.
  '''

  # Get the total number of cities
  n_cities = len(cities_objects)

  # Generate the initial Population
  initial_population = generate_initial_population(population_size, cities_objects)

  # Generate the initial Pheromone Matrix
  pheromone_matrix = initialize_pheromone_matrix(cities_objects, tau_init=0.0)
  # Update the Pheromone Matrix after each new population
  pheromone_matrix = construction(initial_population, pheromone_matrix, evap_rate=evap_rate, alpha=alpha, beta=beta)

  # Get the Distance Matrix
  distance_matrix = get_distance_matrix(cities_objects)


  # Construct n_iterations populations
  for iteration in range(n_iterations):

    # Empty list to store the new population solutions
    new_population = []

    # Construct population_size solutions
    for solution in range(population_size):

      # Generate a random starting city
      random_idx = random.randint(1, n_cities)
      starting_city = [city for city in cities_objects if int(city.name) == random_idx]

      # Initialize the solution
      solution_path = [starting_city[0]]

      # Get the probabilities between the starting city and other cities
      remaining_cities = [city for city in cities_objects if int(city.name) != random_idx]


      # Compute the rest of the path by observing the remaining cities
      while (len(remaining_cities) > 1):

        # Generate a random float (0, 1)
        random_prob = random.random()

        # Get Probabilities for the remaining cities
        probabilities_dict = get_probabilities(starting_city[0], remaining_cities, pheromone_matrix, distance_matrix, alpha=alpha, beta=beta)
        # Get the Cumulative Distributive Function For remaining cities
        cdf_map = get_cdf(probabilities_dict)

        # Look for the matching city
        for city, cdf in cdf_map.items():

          # If a match was found (city with cdf greater than or equal the random probability)
          if random_prob <= cdf:

            next_city = [city_obj for city_obj in cities_objects if int(city_obj.name) == (city+1)]
            solution_path.append(next_city[0])    # Add city to the path

            # Get Remaining cities
            remaining_cities = [city_obj for city_obj in remaining_cities if int(city_obj.name) != (city+1)]

            break   # Exit search, as a match was found

      # Add the final / remaining city
      solution_path.append(remaining_cities[0])

      # Create the path object and append it to the new population
      cities_indicies = [int(city.name)-1 for city in solution_path]
      cost = get_cost(cities_indicies, distance_matrix)
      fitness = 1 / cost
      path_obj = path(cities=solution_path, fitness=fitness, cost=cost)
      new_population.append(path_obj)


    # Update the Pheromone Matrix after each new population
    pheromone_matrix = construction(new_population, pheromone_matrix, evap_rate=evap_rate, alpha=alpha, beta=beta)


  return new_population

In [832]:
population = aco_algorithm(cities_objects, population_size, n_iterations, evap_rate=0.05, alpha=0.6, beta=0.6)
fittest_solution = get_fittest(population)

print('Fittest Solution Is Path:', end=' ')
for city in fittest_solution.cities:
  print(city.name, end=' -> ')

print(f'With Cost = {fittest_solution.cost}')

Fittest Solution Is Path: 2.0 -> 13.0 -> 1.0 -> 11.0 -> 8.0 -> 6.0 -> 3.0 -> 14.0 -> 7.0 -> 5.0 -> 9.0 -> 15.0 -> 10.0 -> 12.0 -> 4.0 -> With Cost = 460.1174029415785
